### Procesamiento de Lenguaje Natural
### Desafio 1

#### Nombre: Eduardo Echeverria

#### Enunciado:

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.

#### Librerias

In [1]:
# Importamos las librerias necesarias
import numpy as np
import pandas as pd
import random
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score
from sklearn.datasets import fetch_20newsgroups

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.

#### Carga de Datos

In [18]:
# Cargamos los datasets de entrrenamiento y prueba
newsgroup_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroup_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

#### Vectorizacion: TF-IDF

In [7]:
# Instanciamos el vectorizador TF-IDF
tf_idf_vect = TfidfVectorizer()

In [8]:
# Realizamos el ajuste y transformación de los datos.
X_train = tf_idf_vect.fit_transform(newsgroup_train.data)
y_train = newsgroup_train.target

In [10]:
# Mostramos el tamaño del dataset, cantidad de documentos y tamaño del vocabulario.
print(type(X_train))
print(f'Tamaño del dataset: {X_train.shape}')
print(f'Cantidad de documentos: {X_train.shape[0]}')
print(f'Tamaño del vocabulario: {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
Tamaño del dataset: (11314, 101631)
Cantidad de documentos: 11314
Tamaño del vocabulario: 101631


#### Similaridad:

Utilizando el vectorizador TF-IDF

In [11]:
# Generamos aleatoriamente 5 números entre el rango correspondiente a la cantidad de documentos.
document_index_list = []
for num in range(5):
    rand_int = random.randint(1, X_train.shape[0])
    document_index_list.append(rand_int)

print(document_index_list)

[7629, 410, 2010, 1885, 3308]


In [15]:
# Calculamos la similaridad Coseno, la clase del documento original y las 5 clases mas similares.
for index in document_index_list:
    print(f"\n~~~~~~~~~~ INICIO DEL DOCUMENTO {index} ~~~~~~~~~~\n")
    print(newsgroup_train.data[index])
    print(f"~~~~~~~~~~ FIN DEL DOCUMENTO {index} ~~~~~~~~~~\n")
    
    # Calculamos la similaridad Coseno
    cos_sim = cosine_similarity(X_train[index], X_train)[0]
    most_similar = np.argsort(cos_sim)[::-1][1:6]
    original_class = newsgroup_train.target_names[y_train[index]]

    # Dezplegamos los resultados
    print(f"Analisis del Documento: {index}\n")
    print(f"Indices de los 5 documentos mas similares: {most_similar}")
    print(f"La clase del documento original es: {original_class}")
    print(f"Las clases de los 5 documentos mas similares son:")

    for i in most_similar:
        print(newsgroup_train.target_names[y_train[i]])


~~~~~~~~~~ INICIO DEL DOCUMENTO 7629 ~~~~~~~~~~

This announcement is somewhat disconcerting; it doesn't do anything
evil in and of itself, but bodes badly for the future of open
algorithms and standards in information security.  I won't start
panicking until/unless DES or RSA or stuff like that is prohibited, but
I'm a little anxious.  (No doubt it won't be long before someone posts
explaining how this just a small part of some far-ranging and
long-lived NSA-PKP-IRS-FBI-CIA-HandgunControlInc-Clinton conspiracy to
subvert freedom, democracy, and mathematics.)  My feeling is that the
administration probably isn't that worried about things like DES and
RSA and PGP and RIPEM, since they'll never be used by a group much
wider than us computer geeks.

The fact that this just came out now suggests one of two things:

1.  The NSA has been working on this for a long time, and it only just
    now happened to be ``ready'' to release to the world at this time.

2.  The NSA has been working on t

#### Análisis

Luego de realizada la prueba se puede observar lo siguiente.
Para el documento 7629, el modelo lo clasifica como de la clase "SCI.CRYPT" es decir como un articulo relacionado con conocimiento científico criptografico, lo cual es correcto dado que menciona conceptos de encriptacion simétrica y asimétrica, así como diferentes algoritmos de encriptación. Tambien se observa que el modelo identifica que todos los articulos mas similares son de la misma clase "SCI.CRYPT", por tanto la clasificación es correcta.

Para el documento 410, el artículo brinda una descripción técnica acerca armas de fuego. El modelo clasifica acertadamente el artículo dentro de la clase "TALK.POLITICS.GUNS". Para los documentos más similares, la mayoría (3 de 5) corresponden a la misma clase, sin embargo tambien aparecen la clase "SCI.SPACE" lo cual puede deberse a algunos términos técnicos. También aparece la clase "ALT.ATHEISM", la cual parece ser una clase mucho menos relacionada con el tema del artículo y su clase original. En general el modelo clasificó bien el docuemnto y los documentos más similares.

El documento 2010, contiene unicamente 4 palabras, lo cual hace difícil clasificarlo acertadamente. El modelo le asigna la clase original de "REC.SPORT.BASEBALL" probablemente por la palabra "borders", sin embargo esta clase no resulta del todo acertada. Entre las clases de los documentos más similares, aparece la clase "SOC.RELIGION.CHRISTIAN", la cual tiene sentido considerando la palabra "blasphemy". En general, para este documento el modelo no realizó una clsificacion acertda, pero nuevamente se debe considerar que el documento contiene muy pocas palabras significativas (unicamente "borders" y "blasphemy" ya que "this" y "on" no proporcionan mucho contexto).

El documento 1885 trata un tema relacionado con substancias endulzantes, los efectos en la salud y las recomendaciones de la FDA de Estados Unidos. El modelo clasifica el artículo dentro de la clase "SCI.MED" lo cual es acertado. Sin embargo se observa que las clases de los documentos más similares son muy variadas, siendo solamente uno de ellos de la misma clase "SCI.MED". Las otras clases, no guardan mucha relación con el tema tratado en el documento.

Finalmente, el documento 3308, si bien es también corto, proporciona el suficiente contexto para entender que se discute acerca del valor de los votos matrimoniales. El modelo clasifica el documento dentro de la clase "SOC.RELIGION.CHRISTIAN" lo cual es acertado. De igual forma, el modelo identifica con la misma clase a todos los documentos mas similares.



##### ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

#### Vectorizador TF-IDF optimizado

In [22]:
# Utilizamos los siguientes parametros para instanciar el vectorizador:
#   sublinear_tf: True
#   ngram_range: (1, 1)
#   max_df: 0.95
#   min_df: 1
#   stop_words: 'english'

tf_idf_vect = TfidfVectorizer(sublinear_tf=True, max_df=0.5, min_df=5, ngram_range=(1, 1), stop_words='english')

In [23]:
# Realizamos el ajuste y transformación de los datos.
X_train_tf = tf_idf_vect.fit_transform(newsgroup_train.data)
X_test_tf = tf_idf_vect.transform(newsgroup_test.data)
y_train_tf = newsgroup_train.target
y_test_tf = newsgroup_test.target

#### Modelo Naïve-Bayes Multinomial

Aplicado al vectorizador TF-IDF

In [27]:
# Definimos los modelos a entrenar. 
naive_bayes_models = {
    'MultinomialNB': MultinomialNB(),
    'ComplementNB': ComplementNB(),
}

# Definimos algunos valores de inicio
best_f1 = 0
best_model = None

for model, model_instance in naive_bayes_models.items():
    model_instance.fit(X_train_tf, y_train_tf)
    y_pred_tf = model_instance.predict(X_test_tf)
    f1 = f1_score(y_test_tf, y_pred_tf, average='macro')

    if f1 > best_f1:
        best_f1 = f1
        best_model = model

print(f"El mejor F1-score es: {best_f1}, obtenido con el modelo {best_model}")

El mejor F1-score es: 0.6820089088033873, obtenido con el modelo ComplementNB


#### Análisis

Luego de probar con diferentes valores para los parámetros del vectorizador, encontramos que los mejores resultados se obtienen con los siguientes parámetros:
- sublinear_tf: True
- ngram_range: (1, 1)
- max_df: 0.95
- min_df: 1
- stop_words: 'english'

La obtención de estos valores fue totalmente experimental.

Se observa también que el modelo que mejores resultados, en cuanto al valor del F1-Score es el "Complement Naïve-Bayes". El mejor valor obtenido es de 0.68


##### ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.

In [28]:
# Utilizamos las mismas configuraciones del vectorizador TF-IDF y Count del primer ejercicio.
tf_idf_vect = TfidfVectorizer(sublinear_tf=True, max_df=0.5, min_df=5, ngram_range=(1, 1), stop_words='english')

In [29]:
# Volvemos a realizar el ajuste y transformación de los datos.
X_tf_idf = tf_idf_vect.fit_transform(newsgroup_train.data)

In [30]:
# Transponemos la matriz Documento-Término para obtener la matriz Término-Documento
transposed_matrix = X_tf_idf.transpose()

print(f"Dimension de la matriz original: {X_tf_idf.shape}")
print(F"Dimension de la matriz transpuesta: {transposed_matrix.shape}")

Dimension de la matriz original: (11314, 17797)
Dimension de la matriz transpuesta: (17797, 11314)


In [31]:
# Obtenemos los feature names, es decir los términos
feature_names = tf_idf_vect.get_feature_names_out()

for feature in feature_names:
    print(feature)

00
000
0000
001
002
01
011
014
02
0200
02106
02139
03
030
0300
033
04
040
05
0500
06
0666
07
08
081
0863
089
09
0_
0a
0b
0c
0d
0f
0g
0h
0i
0iv
0j
0k
0l
0m
0m75u
0n
0o
0p
0q
0qax
0qq
0r
0s
0t
0tbxn
0tbxom
0tq
0u
0v
0w
0x
0y
0z
10
100
1000
100k
101
1010
102
1024
1024x768
1024x768x256
1026
1029
103
1030
1036
104
1047
105
1054
106
1069
107
1070
1074
108
109
1091
10k
10mb
10mhz
10th
10w
11
110
1100
111
1111
1113
112
113
114
1147
115
1150
116
117
1174
118
119
11th
12
120
1200
120mb
121
122
123
124
125
126
127
128
1280
1280x1024
1285
128k
129
12k
12mb
12th
12v
13
130
1300
1304
131
132
133
1333
134
135
136
137
1371
138
139
13p
13th
14
140
1400
141
142
1421
1424
143
1430
144
145
1453
146
147
148
149
14di
14th
15
150
1500
151
152
15213
153
1536
154
1542b
155
156
157
158
159
15k
15th
16
160
1600
1604s
161
162
163
164
16450
165
16550
166
167
168
169
16bit
16k
16m
16mb
16mhz
16th
16v
17
170
171
172
173
174
175
176
177
1776
178
179
17th
18
180
1800
1808
18084tm
181
182
183
184
185
1850
186
187
1876


In [32]:
# Seleccionamos 5 términos. En este caso, términos relacionados con autos y motores
words_to_analyze = ["science", "software", "christianity", "engine", "citizens"]

In [33]:
for word in words_to_analyze:
    index = tf_idf_vect.vocabulary_.get(word)
    if index:
        cosine_sim = cosine_similarity(transposed_matrix[index].reshape(1, -1), transposed_matrix)[0]
        most_sim = np.argsort(cosine_sim)[::-1][1:6]
        print("~"*50)
        print(f"Palabra analizada: {word}")
        print("Palabras mas similares:\n")

        for i in most_sim:
            print(feature_names[i])
    else:
        print(f"\nPalabra: {word} no encontrada")

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Palabra analizada: science
Palabras mas similares:

scientific
cognitivists
behaviorists
scientists
empirical
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Palabra analizada: software
Palabras mas similares:

hardware
pc
packages
maturity
use
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Palabra analizada: christianity
Palabras mas similares:

christian
christians
religion
versed
religions
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Palabra analizada: engine
Palabras mas similares:

exhaust
clamp
rebuilt
oil
household
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Palabra analizada: citizens
Palabras mas similares:

abiding
felony
arrest
government
granting


#### Análisis

Se puede observar lo siguiente:

Para la palabra seleccionada "Science" o ciencia, las palabras mas similares encontradas por el modelo guardan estrecha relación con las palabras más similares.

Para la palabra "Software", el modelo identifica palabras muy relacionadas con el mundo de la computación, sin embargo también identifica palabras como "maturity" y "use" que pueden resultar un poco ambiguas, comparadas con el contexto de la palabra "Software".

La tercera palabra, "Christianity" se encuentra completamente relacionada con las palabras identificadas por el modelo, las cuales son todas relacionadas con el tema religioso cristiano.

La palabra "Engine" o motor, se encuentra también bastante relacionada con las palabras identificadas por el modelo como las más similares. Todas estas palabras se relacionan con maquinaria, vehículos, aceite, etc. Quizás la mas cuestionable es la palabra "household" la cual está mas relacionada con el contexto del hogar, lo cual llega a ser un poco ambiguo en el contexto de la palabra original "Engine".

Finalmente, para la palabra "Citizens" o ciudadanos. El modelo identifica otras palabras que guardan relación con la palabra original, sin embargo esto sucede dependiendo mucho dentro del contexto mas relacionado con asuntos legales o gubernamentales.